# 03 - About $\textsf{RLWE}'$
---

From <a href='https://eprint.iacr.org/2020/086'> Bootstrapping in FHEW-like Cryptosystems </a>

In [1]:
import numpy as np
from note_include.Ring  import Ring
from note_include.RLWE  import RLWE
from note_include.RLWEp import RLWEp
from note_include.utils.gadget_decomposition import gadget_decomposition, gadget_composition, format_ring_list

---
#### Parameters


In [2]:
n   = 16
q   = 7681
B   = 4
d   = 3
std = 3.2

---
Test Gadget (De)Composition

In [3]:
coeffs = [i for i in range(n)]
poly   = Ring(n, q, coeffs)
decomposed_polys = gadget_decomposition(poly, B, int(np.ceil(np.log(q) / np.log(B))) )
composed_poly    = gadget_composition(decomposed_polys, B, q)

print('Plain value : ', coeffs)
print('---- Decomposed polynomials ----')
print(format_ring_list(decomposed_polys))
print('---- Composed polynomials ----')
print(composed_poly)

Plain value :  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
---- Decomposed polynomials ----
R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 0x^4 + 1x^5 + 2x^6 + 3x^7 + 0x^8 + 1x^9 + 2x^10 + 3x^11 + 0x^12 + 1x^13 + 2x^14 + 3x^15)
R(n=16, q=7681, coeffs= 0 + 0x + 0x^2 + 0x^3 + 1x^4 + 1x^5 + 1x^6 + 1x^7 + 2x^8 + 2x^9 + 2x^10 + 2x^11 + 3x^12 + 3x^13 + 3x^14 + 3x^15)
R(n=16, q=7681, coeffs= 0 + 0x + 0x^2 + 0x^3 + 0x^4 + 0x^5 + 0x^6 + 0x^7 + 0x^8 + 0x^9 + 0x^10 + 0x^11 + 0x^12 + 0x^13 + 0x^14 + 0x^15)
R(n=16, q=7681, coeffs= 0 + 0x + 0x^2 + 0x^3 + 0x^4 + 0x^5 + 0x^6 + 0x^7 + 0x^8 + 0x^9 + 0x^10 + 0x^11 + 0x^12 + 0x^13 + 0x^14 + 0x^15)
R(n=16, q=7681, coeffs= 0 + 0x + 0x^2 + 0x^3 + 0x^4 + 0x^5 + 0x^6 + 0x^7 + 0x^8 + 0x^9 + 0x^10 + 0x^11 + 0x^12 + 0x^13 + 0x^14 + 0x^15)
R(n=16, q=7681, coeffs= 0 + 0x + 0x^2 + 0x^3 + 0x^4 + 0x^5 + 0x^6 + 0x^7 + 0x^8 + 0x^9 + 0x^10 + 0x^11 + 0x^12 + 0x^13 + 0x^14 + 0x^15)
R(n=16, q=7681, coeffs= 0 + 0x + 0x^2 + 0x^3 + 0x^4 + 0x^5 + 0x^6 + 0x^7 + 0x^8 +

---
Encryption and Multiplication with Constant

In [4]:
B = 2

rlwep = RLWEp(n, q, std, B)
rlwe  = RLWE(n, q, std)

sk, pk     = rlwe.keygen()
coeffs     = [i for i in range(n)]
big_const  = int(30)
poly       = Ring(n, q, coeffs)

rlwep_ctxts     = rlwep.encrypt(poly, sk)
rlwe_ctxt       = rlwe.encrypt(poly, sk)
ctxt_rlwep_mult = rlwep.mult_constant(rlwep_ctxts, big_const)
ctxt_rlwe_mult  = rlwe.mult_ring_cnst(rlwe_ctxt, big_const)

ptxt_rlwep = rlwe.decrypt(ctxt_rlwep_mult, sk)
ptxt_rlwe  = rlwe.decrypt(ctxt_rlwe_mult,  sk)

print("Multiplier       : ", big_const)
print("Message          : ", poly)
print("Ideal Result     : ", big_const * poly)
print("Plaintext(RLWEp) : ", ptxt_rlwep)
print("Plaintext(RLWE)  : ", ptxt_rlwe)

Multiplier       :  30
Message          :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
Ideal Result     :  R(n=16, q=7681, coeffs= 0 + 30x + 60x^2 + 90x^3 + 120x^4 + 150x^5 + 180x^6 + 210x^7 + 240x^8 + 270x^9 + 300x^10 + 330x^11 + 360x^12 + 390x^13 + 420x^14 + 450x^15)
Plaintext(RLWEp) :  R(n=16, q=7681, coeffs= 7678.0 + 38.0x + 60.0x^2 + 91.0x^3 + 129.0x^4 + 150.0x^5 + 179.0x^6 + 210.0x^7 + 239.0x^8 + 262.0x^9 + 295.0x^10 + 330.0x^11 + 362.0x^12 + 385.0x^13 + 419.0x^14 + 452.0x^15)
Plaintext(RLWE)  :  R(n=16, q=7681, coeffs= 7621.0 + 60.0x + 210.0x^2 + 60.0x^3 + 0.0x^4 + 180.0x^5 + 240.0x^6 + 120.0x^7 + 300.0x^8 + 150.0x^9 + 240.0x^10 + 300.0x^11 + 240.0x^12 + 420.0x^13 + 330.0x^14 + 450.0x^15)


---
Encryption and Multiplication with polynomial

In [6]:
B = 2

rlwep = RLWEp(n, q, std, B)
rlwe  = RLWE(n, q, std)

sk, pk          = rlwe.keygen()
coeffs          = [i for i in range(n)]
big_coeffs      = [30 for _ in range(n)]
poly            = Ring(n, q, coeffs)
poly_multiplier = Ring(n, q, big_coeffs)

rlwep_ctxts     = rlwep.encrypt(poly, sk)
rlwe_ctxt       = rlwe.encrypt(poly, sk)
ctxt_rlwep_mult = rlwep.mult_poly(rlwep_ctxts, poly_multiplier)
ctxt_rlwe_mult  = rlwe.mult_ring_ptxt(rlwe_ctxt, poly_multiplier)

ptxt_rlwep = rlwe.decrypt(ctxt_rlwep_mult, sk)
ptxt_rlwe  = rlwe.decrypt(ctxt_rlwe_mult,  sk)

print("Multiplier       : ", big_const)
print("Message          : ", poly)
print("Ideal Result     : ", poly * poly_multiplier)
print("Plaintext(RLWEp) : ", ptxt_rlwep)
print("Plaintext(RLWE)  : ", ptxt_rlwe)

Multiplier       :  30
Message          :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
Ideal Result     :  R(n=16, q=7681, coeffs= 4081.0 + 4141.0x + 4261.0x^2 + 4441.0x^3 + 4681.0x^4 + 4981.0x^5 + 5341.0x^6 + 5761.0x^7 + 6241.0x^8 + 6781.0x^9 + 7381.0x^10 + 360.0x^11 + 1080.0x^12 + 1860.0x^13 + 2700.0x^14 + 3600.0x^15)
Plaintext(RLWEp) :  R(n=16, q=7681, coeffs= 4088.0 + 4142.0x + 4250.0x^2 + 4416.0x^3 + 4652.0x^4 + 4970.0x^5 + 5316.0x^6 + 5708.0x^7 + 6220.0x^8 + 6782.0x^9 + 7362.0x^10 + 347.0x^11 + 1063.0x^12 + 1849.0x^13 + 2689.0x^14 + 3593.0x^15)
Plaintext(RLWE)  :  R(n=16, q=7681, coeffs= 4381.0 + 4441.0x + 4561.0x^2 + 4741.0x^3 + 4981.0x^4 + 4981.0x^5 + 5281.0x^6 + 5761.0x^7 + 6001.0x^8 + 6601.0x^9 + 7261.0x^10 + 420.0x^11 + 960.0x^12 + 1620.0x^13 + 2340.0x^14 + 3420.0x^15)
